In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy
import sys
sys.path.append("..")
import os
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import json
from IPython import display

from tqdm import tqdm, tqdm_notebook, tqdm_pandas
tqdm.pandas()

import copy

from Scripts.saving import submit

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from datetime import datetime


from Scripts.utils import *

import json

import featuretools as ft

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

Populating the interactive namespace from numpy and matplotlib


In [4]:
base_dir = ".."
input_dir = open( os.path.join(base_dir, "datadir.txt"), "r").read()[:-1]
pred_dir = os.path.join(base_dir, "predictions")
processed_dir = os.path.join(input_dir, "processed")
generate_dir = os.path.join(input_dir, "generate")

In [5]:
train = pd.read_csv(
    os.path.join(processed_dir, "train_df_0.814.csv"), index_col=0
)
test = pd.read_csv(
    os.path.join(processed_dir, "test_df_0.814.csv"), index_col=0
)
y_train = pd.read_csv( os.path.join(input_dir,"processed", 'train_all_JSON.csv'), index_col=0 ).skilled

In [6]:
train_timeseries_no_zeros = pd.read_csv(
        os.path.join(processed_dir, "train_timeseries_no_zeros.csv"), index_col=0
    )
train_timeseries_no_zeros.drop(
            [c for c in train_timeseries_no_zeros.columns if c.endswith("mode")],
            axis=1,
            inplace=True
)
train_ability_upgrades = pd.read_csv(
        os.path.join(processed_dir, "train_ability_upgrades.csv"), index_col=0
    )
train_damage_targets = pd.read_csv(
        os.path.join(processed_dir, "train_damage_targets.csv"), index_col=0
    )
train_team = pd.read_csv(
        os.path.join(processed_dir, "train_team.csv"), index_col=0
    )
train_purchase = pd.read_csv(
        os.path.join(processed_dir, "train_item_purchase_log.csv"), index_col=0
    )

In [7]:
test_timeseries_no_zeros = pd.read_csv(
        os.path.join(processed_dir, "test_timeseries_no_zeros.csv"), index_col=0
    )
test_timeseries_no_zeros.drop(
            [c for c in test_timeseries_no_zeros.columns if c.endswith("mode")],
            axis=1,
            inplace=True
)
test_ability_upgrades = pd.read_csv(
        os.path.join(processed_dir, "test_ability_upgrades.csv"), index_col=0
    )
test_damage_targets = pd.read_csv(
        os.path.join(processed_dir, "test_damage_targets.csv"), index_col=0
    )
test_team = pd.read_csv(
        os.path.join(processed_dir, "test_team.csv"), index_col=0
    )
test_purchase = pd.read_csv(
        os.path.join(processed_dir, "test_item_purchase_log.csv"), index_col=0
    )

In [ ]:
train_dfs = [
    #(train_timeseries_no_zeros, "timeseries_no_zeros" ),
    #(train_ability_upgrades, "ability_upgrades"),
    #(train_damage_targets, "damage_targets"),
    (train_purchase, "purchase"),
]

In [ ]:
test_dfs = [
    #(test_timeseries_no_zeros, "timeseries_no_zeros" ),
    #(test_ability_upgrades, "ability_upgrades"),
    #(test_damage_targets, "damage_targets"),
    (test_purchase, "purchase"),
]

In [ ]:
def fix_index(df):
    index = df.index
    df["id"] = index
    return df.set_index("id")

In [ ]:
for i in tqdm( range(len(train_dfs)) ):
    temp_train = fix_index( train_dfs[i][0] )
    temp_test = fix_index( test_dfs[i][0] )
    df_name = train_dfs[i][1]
    
    print("-"*30)
    print(df_name)
    print("-"*30)
    
    train_gen, test_gen = generate_for_train_test(
                            train = temp_train, 
                            test = temp_test,
                    
                            shuffle=True,
                            trans_primitives=['divide_numeric', 'multiply_numeric', 'isin', 'diff'],
                            var_types={}, 
                            N_FEATURES=1500, 
    )
    train_gen = pd.concat([temp_train, train_gen],axis=1)
    test_gen = pd.concat([temp_test, test_gen],axis=1)
    
    # Select columns in train and test
    in_train_and_test = list(
        set(train_gen.columns).intersection( set(test_gen.columns) )
    )
    train_gen = train_gen[in_train_and_test]
    test_gen = test_gen[in_train_and_test]
    
    best_features = get_most_importance(train_gen, y_train, n_inp = 400)
    
    date = datetime.now()
    suffix = f"[{date.day}.{date.month}]{date.hour}:{date.minute}"
    
    save_train_test(train_gen[best_features], test_gen[best_features], base_name=df_name+suffix, path=generate_dir)
    

  0%|          | 0/1 [00:00<?, ?it/s]

------------------------------
purchase
------------------------------
---------------
Entityset: players
  Entities:
    train_players [Rows: 99871, Columns: 469]
  Relationships:
    No relationships
Start dfs


In [ ]:
list_prim = ft.primitives.list_primitives()
list_prim[list_prim.type == "transform"]

In [85]:
model = cb.CatBoostClassifier()

feature_score = pd.DataFrame(list(
                                zip(
                                    train_gen.dtypes.index, 
                                    model.get_feature_importance(
                                                                cb.Pool(train_gen, label=y_train)
                                                                )
                                    )
                                ),
                columns=['Feature','Score'])

In [89]:
train_gen.dtypes.index.shape

(1585,)